# 7일차 과제 : PCA
# 데이터 표준화 이후, 주성분 분석을 실시하여 2차원(1 vs 2 주성분, 1 vs 3 주성분, …) 산점도를 그려라.(목표변수로 색 구분), + 주성분 naming
# 데이터 : 유방암 데이터(변수: 31개, 자료 수: 569개)
# -.목표변수(diagnosis, 0: Malignant(악성,212개), 1: Benign(양성,357개)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

In [ ]:
ds_supervisor=pd.read_csv('./Downloads/week4day1/직무능력평가.csv',engine='python')
ds_supervisor_x=ds_supervisor.drop("EVAL",axis=1)
ds_supervisor_y=ds_supervisor["EVAL"]
ds_supervisor_x.head()


In [ ]:
ds_supervisor_x.corr()

In [ ]:
sns.pairplot(ds_supervisor_x)

In [ ]:
scaler=StandardScaler()
ds_supervisor_x_scaled=scaler.fit_transform(ds_supervisor_x)
print("Scaled Data:\n {}".format(ds_supervisor_x_scaled[:5].round(3)))
print("Shape:\n {}".format(ds_supervisor_x_scaled.shape))

In [ ]:
pca=PCA(n_components=6)
x_pca=pca.fit_transform(ds_supervisor_x_scaled)
print("PCA Data :\n {}".format(x_pca[:5]))
print("Shape :\n {}".format(x_pca.shape))
ds_x_pca=pd.DataFrame(x_pca,columns=["Prin1","Prin2","Prin3","Prin4","Prin5","Prin6"])


In [ ]:
ds_pca=pd.DataFrame()
ds_pca["Prin"]=[I for I in range(1,7)]
ds_pca["EigenValues"]=pca.explained_variance_ ## Eigen Values
ds_pca["EigenValueRatio"]=pca.explained_variance_ratio_ ## Eigen Value ratio
ds_pca["CumsumEigenValueRatio"]=np.cumsum(pca.explained_variance_ratio_)
ds_pca.round(3)

In [ ]:
ds_feature=pd.DataFrame({"Feature":ds_supervisor_x.columns})
ds_eigenvectors=pd.concat([ds_feature,
    pd.DataFrame(pca.components_, columns=["Prin1","Prin2","Prin3","Prin4","Prin5","Prin6"])],axis=1)
ds_eigenvectors

In [ ]:
plt.plot(ds_pca["Prin"], ds_pca["EigenValueRatio"], label="Ratio")
plt.plot(ds_pca["Prin"], ds_pca["CumsumEigenValueRatio"], label="Cumsum Ratio")
plt.ylabel("Eigen Value ratio")
plt.xlabel("Prin")
plt.legend(loc="best")
ds_pca

In [ ]:
sns.pairplot(ds_x_pca)
ds_x_pca.corr().round(3)

In [ ]:
x_var=["Prin1","Prin2"]
ds_x_pca_const=sm.add_constant(ds_x_pca[x_var])
X_train, X_test, y_train, y_test=train_test_split(ds_x_pca_const, ds_supervisor_y,
                                                 test_size=0.3, random_state=2222)
reg_model=sm.OLS(y_train, X_train)
reg_model=reg_model.fit()
print(reg_model.summary())

#분석자료수 22
#잔차자유도 18
#모델자유도 2

#ANOVA의 p값이 유의수준 0.05보다 작으므로 모델 유의

#설명력 0.65

#회귀계수 검정
#Prin2의 p값이 0.05보다는 크므로 유의수준 0.05에서 유의하지는 않지만
#유의수준이 0.1이라면 해당 회귀계수는 유의
